### V Ivanouskaya
21.04.2024
homework PyTorch
v1

Решить задачу на Торче
Решить задачу просрочки кредита используя для прогназирования нейронные сети. Посмотреть работу с несбалансированными данными.
Набор данных представляет собой исторические данные по 251503 заемщикам (https://www.kaggle.com/c/GiveMeSomeCredit/data ).
Задача является бинарной классификацией.
Цель - предсказать будет ли тот или иной заемщик испытывать финансовые трудности в ближайшие 2 года, т.е. будет ли просрочка по займу.
Выборка разделена на тренировочную и тестовую ( 150000 в тренировочной части, 101503 в тестовой).

In [17]:
from google.colab import files
import os

folder_path = '/content/credit/'
os.makedirs(folder_path, exist_ok=True)
uploaded = files.upload()
for file_name in uploaded.keys():
    with open(os.path.join(folder_path, file_name), 'wb') as f:
        f.write(uploaded[file_name])
print("Файлы в папке:", os.listdir(folder_path))

Загрузите файлы в папку: /content/credit/


Saving cs-test.csv to cs-test.csv
Saving cs-training.csv to cs-training.csv
Файлы в папке: ['cs-test.csv', 'cs-training.csv']


In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [21]:
train_data = pd.read_csv('cs-training.csv')
test_data = pd.read_csv('cs-test.csv')

### Подготовка данных

In [22]:
train_data.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [23]:
###### RevolvingUtilizationOfUnsecuredLines: Процент использования кредитных линий.
######  age: Возраст заемщика.
######  NumberOfTime30-59DaysPastDueNotWorse: Количество раз, когда заемщик опаздывал с платежом на 30-59 дней.
######  DebtRatio: Долговая нагрузка.
######  MonthlyIncome: Ежемесячный доход.
###### NumberOfOpenCreditLinesAndLoans: Количество открытых кредитных линий и займов.
###### NumberOfTimes90DaysLate: Количество раз, когда заемщик опаздывал с платежом более чем на 90 дней.
###### NumberRealEstateLoansOrLines: Количество ипотечных кредитов и кредитных линий.
###### NumberOfTime60-89DaysPastDueNotWorse: Количество раз, когда заемщик опаздывал с платежом на 60-89 дней.
###### NumberOfDependents: Количество иждивенцев.

In [24]:
# Удаление лишнего столбца
train_data.drop(columns=['Unnamed: 0'], inplace=True)

In [25]:
train_data.isnull().sum()

SeriousDlqin2yrs                            0
RevolvingUtilizationOfUnsecuredLines        0
age                                         0
NumberOfTime30-59DaysPastDueNotWorse        0
DebtRatio                                   0
MonthlyIncome                           29731
NumberOfOpenCreditLinesAndLoans             0
NumberOfTimes90DaysLate                     0
NumberRealEstateLoansOrLines                0
NumberOfTime60-89DaysPastDueNotWorse        0
NumberOfDependents                       3924
dtype: int64

In [26]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      150000 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  150000 non-null  float64
 2   age                                   150000 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  150000 non-null  int64  
 4   DebtRatio                             150000 non-null  float64
 5   MonthlyIncome                         120269 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       150000 non-null  int64  
 7   NumberOfTimes90DaysLate               150000 non-null  int64  
 8   NumberRealEstateLoansOrLines          150000 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  150000 non-null  int64  
 10  NumberOfDependents                    146076 non-null  float64
dtype

In [27]:
# Обработка пропущенных значений
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
train_data[['MonthlyIncome', 'NumberOfDependents']] = imputer.fit_transform(train_data[['MonthlyIncome', 'NumberOfDependents']])

In [29]:
# Нормализация данных, кроме целевой переменной
scaler = StandardScaler()
X = train_data.drop(columns=['SeriousDlqin2yrs'])
y = train_data['SeriousDlqin2yrs']
X_scaled = scaler.fit_transform(X)

In [30]:
# Проверка баланса классов
class_balance = y.value_counts(normalize=True)
class_balance

SeriousDlqin2yrs
0    0.93316
1    0.06684
Name: proportion, dtype: float64

In [31]:
# Разделение данных на тренировочную и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
# Проверка размеров полученных выборок
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((120000, 10), (30000, 10), (120000,), (30000,))

In [32]:
# Вычисление весов классов для учета несбалансированности
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

In [33]:
# Вычисление весов классов для учета несбалансированности
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [35]:
class CreditModel(nn.Module):
    def __init__(self):
        super(CreditModel, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

###Построение модели

In [37]:
# конвертация данных в тензоры
X_train_tensor = torch.tensor(X_train.astype(np.float32))
y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).view(-1, 1)
X_val_tensor = torch.tensor(X_val.astype(np.float32))
y_val_tensor = torch.tensor(y_val.values.astype(np.float32)).view(-1, 1)
# создание DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [39]:
model = CreditModel()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.view(-1)
        loss = criterion(outputs, targets.squeeze())
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        outputs_val = model(X_val_tensor)
        outputs_val = outputs_val.view(-1)
        loss_val = criterion(outputs_val, y_val_tensor.squeeze())
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Val Loss: {loss_val.item()}')

Epoch 1, Loss: 0.0679347887635231, Val Loss: 0.1885441392660141
Epoch 2, Loss: 0.26521724462509155, Val Loss: 0.1874464899301529
Epoch 3, Loss: 0.1412360817193985, Val Loss: 0.18780001997947693
Epoch 4, Loss: 0.27839362621307373, Val Loss: 0.18749842047691345
Epoch 5, Loss: 0.16370265185832977, Val Loss: 0.1880044788122177
Epoch 6, Loss: 0.24384543299674988, Val Loss: 0.18878820538520813
Epoch 7, Loss: 0.13394826650619507, Val Loss: 0.18670502305030823
Epoch 8, Loss: 0.19281801581382751, Val Loss: 0.1879078596830368
Epoch 9, Loss: 0.05219658091664314, Val Loss: 0.18715283274650574
Epoch 10, Loss: 0.2639848291873932, Val Loss: 0.18961361050605774


###Оценка модели

In [40]:
outputs_val_np = outputs_val.numpy().squeeze()
y_val_pred = (outputs_val_np > 0.5).astype("int32")
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)
roc_auc = roc_auc_score(y_val, outputs_val_np)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 score: {f1}')
print(f'ROC AUC: {roc_auc}')

Precision: 0.5582047685834503
Recall: 0.19850374064837906
F1 score: 0.29286239882266374
ROC AUC: 0.836393477816443


**Precision**:
модель права примерно в 55.8% случаев, что не очень высокий результат.
**Recall**:
модель обнаруживает только около 19.9% всех реальных положительных случаев.
**F1 Score**:
низкое значение F1 говорит о том, что низкая полнота.
**ROC AUC**:
0.836 — это сильный показатель. Значение близкое к 1.0 указывает на высокую способность модели дифференцировать классы.